<a href="https://colab.research.google.com/github/JasonD28/CS-175/blob/master/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final Project Report for CS 175, Spring 2020
**Project Title:** Water Filled Lung Detector

**Project Number:** Group 14

**Student Name(s)**

Allan Tran, 61735904. allannt@uci.edu

Jason Davis, 22336416, jasonbd@uci.edu

Eva Dai, 94015611, ydai8@uci.edu

First, load any source code we need and import them.

In [1]:
%%bash

# Using PyTorch helper functions to simplify training
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.5.1

cp references/detection/utils.py ../
cp references/detection/engine.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/coco_utils.py ../

Your branch is up to date with 'origin/v0.5.1'.


fatal: destination path 'vision' already exists and is not an empty directory.
Already on 'v0.5.1'


In [2]:
%%bash

git clone https://github.com/cocodataset/cocoapi.git
cd cocoapi
cp PythonAPI/pycocotools D:/Python368/dist-packages/ -r

fatal: destination path 'cocoapi' already exists and is not an empty directory.


In [3]:
import torch
import utils
from torch import optim
import torchvision
from src.dataset import PneumoniaDataset
#from engine import train_one_epoch, evaluate
#from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
#from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from src.classifier import Classifier

import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import sampler
from torch.nn.functional import relu as Relu
from torch import sigmoid

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

import timeit

import pandas as pd

## Classifier:

Here we start to load the classifier model and train it with a small sample of our data for demostration purpose.

In [4]:
root = "./data"

In [5]:
model = Classifier();
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

Classifier(
  (convInput): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv32): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv32to64): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv64): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv64to128): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv128): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pooling): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linearto128): Linear(in_features=2097152, out_features=128, bias=True)
  (linearto1): Linear(in_features=128, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [6]:
dataset = PneumoniaDataset(root, True)
validation_dataset = PneumoniaDataset(root, True)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:20])
validation_dataset = torch.utils.data.Subset(validation_dataset, indices[20:])

data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=1, shuffle=True, num_workers=1,
        collate_fn=utils.collate_fn)

validation_data_loader = torch.utils.data.DataLoader(
    validation_dataset, batch_size=1, shuffle=False, num_workers=2,
    collate_fn=utils.collate_fn)


test_dataset = PneumoniaDataset(root, True, 'test')

test_data_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=1, shuffle=False, num_workers=2,
    collate_fn=utils.collate_fn)

optimizer = optim.Adam(model.parameters(), lr=1e-3)
gpu_dtype = torch.cuda.FloatTensor
loss_fn = nn.BCEWithLogitsLoss().type(gpu_dtype)

In [7]:
def train(train_data, dtype, model, loss_fn, optimizer, num_epochs=1):
    for epoch in range(num_epochs):
        print('Starting epoch %d / %d' % (epoch + 1, num_epochs))
        model.train()
        for t, (image, target) in enumerate(train_data):
            imgs = torch.tensor([img.numpy() for img in image], dtype=torch.float32)
            x_var = Variable(imgs.type(dtype))
            l = []
            for tar in target:
              if len(tar['labels']) > 1:
                # temp = [list(tar['labels'])[0]]
                l.append([1])
              else:
                if tar['labels'] == 2:
                  l.append([1])
                else:
                  l.append([0])
            y_var = torch.tensor(l).cuda() # Variable(labels.type(dtype).cuda())
            scores = model(imgs.cuda())
            loss = loss_fn(scores, y_var.float())
            if (t + 1) % 5 == 0:
              print('t = %d, loss = %.4f' % (t + 1, loss.item()))
              print(y_var)
              print(scores)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

In [8]:
train(data_loader, gpu_dtype, model, loss_fn, optimizer, 1)

Starting epoch 1 / 1
t = 5, loss = 0.0010
tensor([[0]], device='cuda:0')
tensor([[-6.9146]], device='cuda:0', grad_fn=<AddmmBackward>)
t = 10, loss = 0.0000
tensor([[0]], device='cuda:0')
tensor([[-26.1347]], device='cuda:0', grad_fn=<AddmmBackward>)
t = 15, loss = 0.0000
tensor([[0]], device='cuda:0')
tensor([[-21.0783]], device='cuda:0', grad_fn=<AddmmBackward>)
t = 20, loss = 0.3503
tensor([[0]], device='cuda:0')
tensor([[-0.8688]], device='cuda:0', grad_fn=<AddmmBackward>)


After the training, we can check the accuracy on the validation set.

In [9]:
def check_accuracy(model, loader):
    """if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')"""  
    num_correct = 0
    num_samples = 0
    model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    for t, (image, target) in enumerate(loader):
        with torch.no_grad():
          imgs = torch.tensor([img.numpy() for img in image], dtype=torch.float32)
          l = []
          metadata = []
          for tar in target:
            if len(tar['labels']) > 1:
              l.append([1])
            else:
              if tar['labels'] == 2:
                l.append([1])
              else:
                l.append([0])
            metadata.append([tar['position']])
          l = torch.tensor(l)
        raw_scores = sigmoid(model(imgs.cuda()))
        scores = torch.tensor([int(raw_scores > 0.37)])
        num_correct += (scores == l).sum()
        num_samples += scores.size(0)
        if (t + 1) % 1 == 0:
            print('t = %d, num_correct = %d, num_samples = %d' % (t + 1, num_correct, num_samples))
    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [10]:
check_accuracy(model, validation_data_loader)

t = 1, num_correct = 0, num_samples = 1
t = 2, num_correct = 1, num_samples = 2
t = 3, num_correct = 2, num_samples = 3
t = 4, num_correct = 2, num_samples = 4
t = 5, num_correct = 3, num_samples = 5
t = 6, num_correct = 4, num_samples = 6
Got 4 / 6 correct (66.67)


Now we can save the predictions into a csv file. But for demostration, here we'll print out the predictions instead.

In [11]:
def save_preds(model, loader, dataset):
    model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    pIds = []
    predictions = []
    for index, (image, target) in enumerate(loader):
        with torch.no_grad():
          imgs = torch.tensor([img.numpy() for img in image], dtype=torch.float32)
        scores =  torch.tensor([int(sigmoid(model(imgs.cuda())) > 0.37)])
        patientId = dataset.imgs[index][:-4]
        p = scores.numpy()[0]
        pIds.append(patientId)
        predictions.append(p)

    d = {'patientId': pIds, 'pred': predictions}
    #df = pd.DataFrame(data=d)
    #path = 'drive/My Drive/cs-175-project/predictions/classifier_w_metadata_prediction.csv'
    #df.to_csv(path, index=False)
    print(d)
    

In [12]:
save_preds(model, test_data_loader, test_dataset)

{'patientId': ['0000a175-0e68-4ca4-b1af-167204a7e0bc', '0005d3cc-3c3f-40b9-93c3-46231c3eb813', '000686d7-f4fc-448d-97a0-44fa9c5d3aa6', '000e3a7d-c0ca-4349-bb26-5af2d8993c3d', '00100a24-854d-423d-a092-edcf6179e061', '0015597f-2d69-4bc7-b642-5b5e01534676', '001b0c51-c7b3-45c1-9c17-fa7594cab96e', '0022bb50-bf6c-4185-843e-403a9cc1ea80', '00271e8e-aea8-4f0a-8a34-3025831f1079', '0028450f-5b8e-4695-9416-8340b6f686b0'], 'pred': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


As shown in the validation and test predictions above, we can see the accuracy is very low. It is caused by the small number of the training data we have in this submission. During the actual project, we ran on the entire dataset of 26k images, which would result in a much higher accuracy of 90.1%.